In [26]:
class CSP:
    def __init__(self, variables, dominios, restricciones):
        """
        Inicializa un problema de satisfacción de restricciones.
        
        Args:
            variables: Lista de variables
            dominios: Diccionario con los dominios de cada variable
            restricciones: Función que verifica si una asignación satisface las restricciones
        """
        self.variables = variables
        self.dominios = dominios
        self.restricciones = restricciones
        self.vecinos = {var: [] for var in variables}
        
        for var in variables:
            for other_var in variables:
                if var != other_var:
                    self.vecinos[var].append(other_var)

def backtracking(assignment, csp):
    """
    Algoritmo de backtracking para resolver un CSP.
    
    Args:
        assignment: Diccionario con la asignación actual (variable -> valor)
        csp: Objeto CSP
        
    Returns:
        Una asignación completa o fallo (None)
    """
    if len(assignment) == len(csp.variables):
        return assignment  
    var = seleccionar_variable_no_asignada(csp.variables, assignment, csp)
    for valor in orden_valores_dominio(var, assignment, csp):
        if es_consistente(var, valor, assignment, csp):
            assignment[var] = valor
            resultado = backtracking(assignment, csp)
            if resultado is not None:
                return resultado
            del assignment[var]
    return None

def seleccionar_variable_no_asignada(variables, assignment, csp):
    """
    Selecciona una variable no asignada usando la heurística MRV (Minimum Remaining Values).
    
    Returns:
        La variable no asignada con el menor número de valores válidos en su dominio
    """
    variables_no_asignadas = [var for var in variables if var not in assignment]
    return min(variables_no_asignadas, key=lambda var: len(csp.dominios[var]))

def orden_valores_dominio(var, assignment, csp):
    """
    REVISAR
    Ordena los valores del dominio según una heurística.
    Por simplicidad, devolvemos el dominio sin ordenar.
    
    Returns:
        Lista de valores en el dominio de la variable
    """
    return csp.dominios[var]

def es_consistente(var, valor, assignment, csp):
    """
    REVISAR
    Verifica si la asignación de valor a var es consistente con las restricciones.
    
    Returns:
        True si la asignación es consistente, False en caso contrario
    """
    assignment_temp = assignment.copy()
    assignment_temp[var] = valor
    return csp.restricciones(assignment_temp)

def AC3(csp):
    """
    Algoritmo AC-3 para consistencia de arcos.
    """

    cola = [(xi, xj) for xi in csp.variables for xj in csp.vecinos[xi]]
    while cola:
        xi, xj = cola.pop(0)
        if borrar_valores_inconsistentes(xi, xj, csp):
            if len(csp.dominios[xi]) == 0:
                return False
            for xk in csp.vecinos[xi]:
                if xk != xj:
                    cola.append((xk, xi))
    return True

def borrar_valores_inconsistentes(xi, xj, csp):
    """
    REVISAR
    Borra valores inconsistentes del dominio de xi con respecto a xj.
    
    Args:
        xi, xj: Variables
        csp: Objeto CSP
        
    Returns:
        True si se borró algún valor, False en caso contrario
    """
    borrado = False
    valores_a_borrar = []
    for x in csp.dominios[xi]:
        satisface_restriccion = False
        for y in csp.dominios[xj]:
            assignment_temp = {xi: x, xj: y}
            if csp.restricciones(assignment_temp):
                satisface_restriccion = True
                break
        if not satisface_restriccion:
            valores_a_borrar.append(x)
            borrado = True
    for valor in valores_a_borrar:
        csp.dominios[xi].remove(valor)
    
    return borrado

In [ ]:

def cursos_lcc_cps():
    variables = [
        # 2DO SEMESTRE
        'Calculo II',
        'Algebra lineal I',
        'Mecanica I',
        'Matematicas discretas',
        'Programacion de computadoras',
        'Curso de tronco comun'
        'Optativa humanidades', 

        # 4TO SEMESTRE
        'Probabilidad',
        'Electromagnetismo',
        'Estructura de datos',
        'Ingenieria de software I',
        'Teoria de la computacion',
        'Curso de tronco comun',
        'Optativa humanidades',

        # 6TO SEMESTRE
        'IA',
        'Arquitectura de computadoras',
        'Analisis de algoritmos',
        'Bases de datos',
        'Especializante I',
        'Integrador I',

        # 8VO SEMESTRE
        'Especializante IV',
        'Especializante V',
        'Integrador III'
    ]
    
    # Dominio: pares (aula, hora)
    aulas = ['3K4-101', '3K4-102', '3K4-103', '3K4-202', '3K4-203']
    horas = ['7:00', '8:00', '9:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00']
    
    dominio = {}
    for curso in variables:
        dominio[curso] = [(aula, hora) for aula in aulas for hora in horas]
    
    def cursos_restricciones(assignment):
        asignados = list(assignment.items())        
        for i in range(len(asignados)):
            curso1, (aula1, hora1) = asignados[i]
            for j in range(i + 1, len(asignados)):
                curso2, (aula2, hora2) = asignados[j]
                if aula1 == aula2 and hora1 == hora2:
                    return False
        
        return True
    
    # Crear el objeto CSP
    csp = CSP(variables, dominio, cursos_restricciones)
    
    AC3(csp)    
    solucion = backtracking({}, csp)
    
    if solucion:
        # print("\nSolución encontrada:")
        # for curso, (aula, hora) in solucion.items():
        #     print(f"{curso}: Aula {aula}, Hora {hora}")
        
        # # Verificar solución
        # print("\nVerificando solución...")
        # is_valid = csp.restricciones(solucion)
        # print(f"La solución es válida: {is_valid}")
        
        # Mostrar horario en formato tabla
        print("\nHorario:")
        horario = {hora: {aula: None for aula in aulas} for hora in horas}
        
        for curso, (aula, hora) in solucion.items():
            horario[hora][aula] = curso
        
        print("    | " + " | ".join(f"Aula {aula}" for aula in aulas))
        print("-" * 50)
        
        for hora in horas:
            row = f"{hora} | "
            for aula in aulas:
                curso = horario[hora][aula]
                if curso:
                    # Abreviamos los nombres para que quepan en la tabla
                    abreviatura = "".join(word[0] for word in curso.split())
                    row += f"{abreviatura}     | "
                else:
                    row += "Libre  | "
            print(row)
    else:
        print("No se encontró solución")

# Ejecutar el problema
if __name__ == "__main__":
    cursos_lcc_cps()


Horario:
    | Aula 3K4-101 | Aula 3K4-102 | Aula 3K4-103 | Aula 3K4-202 | Aula 3K4-203
--------------------------------------------------
7:00 | CI     | IdsI     | II     | Libre  | Libre  | 
8:00 | AlI     | Tdlc     | EI     | Libre  | Libre  | 
9:00 | MI     | Cdtc     | EV     | Libre  | Libre  | 
10:00 | Md     | Oh     | II     | Libre  | Libre  | 
11:00 | Pdc     | I     | Libre  | Libre  | Libre  | 
12:00 | Cdtch     | Adc     | Libre  | Libre  | Libre  | 
13:00 | P     | Ada     | Libre  | Libre  | Libre  | 
14:00 | E     | Bdd     | Libre  | Libre  | Libre  | 
15:00 | Edd     | EI     | Libre  | Libre  | Libre  | 


In [ ]:
cursos_segundo_semestre = [
    'Calculo II',
    'Algebra lineal I',
    'Mecanica I',
    'Matematicas discretas',
    'Programacion de computadoras',
    'Curso de tronco comun',
    'Optativa humanidades'
]

cursos_cuarto_semestre = [
    'Probabilidad',
    'Electromagnetismo',
    'Estructura de datos',
    'Ingenieria de software I',
    'Teoria de la computacion',
    'Curso de tronco comun',
    'Optativa humanidades'
]

cursos_sexto_semestre = [
    'IA',
    'Arquitectura de computadoras',
    'Analisis de algoritmos',
    'Bases de datos',
    'Especializante I',
    'Integrador I'
]

cursos_octavo_semestre = [
    'Especializante IV',
    'Especializante V',
    'Integrador III'
]

# Crear una lista de cursos total
variables = cursos_segundo_semestre + cursos_cuarto_semestre + cursos_sexto_semestre + cursos_octavo_semestre


['Calculo II', 'Algebra lineal I', 'Mecanica I', 'Matematicas discretas', 'Programacion de computadoras', 'Curso de tronco comun', 'Optativa humanidades', 'Probabilidad', 'Electromagnetismo', 'Estructura de datos', 'Ingenieria de software I', 'Teoria de la computacion', 'Curso de tronco comun', 'Optativa humanidades', 'IA', 'Arquitectura de computadoras', 'Analisis de algoritmos', 'Bases de datos', 'Especializante I', 'Integrador I', 'Especializante IV', 'Especializante V', 'Integrador III']
